# Training Decision Transformers to interact with Halfcheetah environment

The Decision Transformer model was introduced by [“Decision Transformer: Reinforcement Learning via Sequence Modeling”](https://arxiv.org/abs/2106.01345) by Chen L. et al. It abstracts Reinforcement Learning as a conditional-sequence modeling problem. The fundamental idea is to use a sequence modeling algorithm, using Transformers, instead of training a policy with usual reinforcement learning models. Given the desired return, past states, and actions, the Transformer will generate future actions to achieve this desired return. It’s an autoregressive model conditioned on the desired return, past states, and actions to generate future actions that achieve the desired return. This is a complete shift in the Reinforcement Learning paradigm since we use generative trajectory modeling (modeling the joint distribution of the sequence of states, actions, and rewards) to replace conventional RL algorithms. It means that in Decision Transformers, we don’t maximize the return but rather generate a series of future actions that achieve the desired return.

In this example we employ the capabilities of the HuggingFace library that has a particular class for this task.The process is the following:



1.   We feed the last K timesteps into the Decision Transformer with three inputs:
      *   Return-to-go
      *   State
      *   Action
2.  The tokens are embedded either with a linear layer if the state is a vector or a CNN encoder if it’s frames.

3.  The inputs are processed by a GPT-2 model, which predicts future actions via autoregressive modeling.

Specifically in our case we don't actually learn by interacting with environment but we learn from data collected by other agents or humans. This approach is usually called offline RL.

If you want to train it, is suggest the use of a GPU.

- `Hardware Accelerator > GPU`

### Install dependencies

### Install and import the packages



In [ ]:
!apt-get update
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    patchelf \
    xvfb

In [ ]:
!pip install gym==0.21.0
!pip install free-mujoco-py
!pip install mujoco
!pip install gym[mujoco]
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets==2.14.5
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.5
!pip install colabgymrender==1.0.2
!pip install xvfbwrapper
!pip install huggingface_hub
!pip install pyarrow==14.0.2
!pip install requests==2.31.0

In [ ]:
!pip install gym==0.21.0
!pip install free-mujoco-py
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install imageio==2.9.0
!pip install imageio-ffmpeg==0.4.5
!pip install colabgymrender==1.0.2
!pip install xvfbwrapper
!pip install huggingface_hub
!pip install pyarrow==14.0.2
!pip install requests==2.31.0

In [ ]:
!pip install mujoco

In [ ]:
!pip install gym[mujoco]

In [ ]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from datasets import load_dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments

### Loading the dataset from the HuggingFace Hub and instantiating the model

HuggingFace repository is huge and for this example we will be training with the halfcheetah “expert” dataset, hosted on hub.

First we need to import the load_dataset function from the HuggingFace datasets package and download the dataset to our machine.

In [ ]:
os.environ["WANDB_DISABLED"] = "true" # we diable weights and biases logging for this tutorial
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")

### Defining a custom DataCollator for the transformers Trainer class

In this case we wish to match the [author's implementation](https://github.com/kzl/decision-transformer), that is we need to perform some additional processing to the data as follows:

* Normalize each feature by subtracting the mean and dividing by the standard deviation.
* Pre-compute discounted returns for each trajectory.
* Scale the rewards and returns by a factor of 1000.
* Augment the dataset sampling distribution so it takes into account the length of the expert agent’s trajectories.

In order to perform this dataset preprocessing, we will use a custom HuggingFace [Data Collator](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#data-collator).
Data Collator are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.

To be able to build batches, data collators may apply some processing (like padding). Some of them (like DataCollatorForLanguageModeling) also apply some random data augmentation (like random masking) on the formed batch.

In [ ]:
@dataclass
class DecisionTransformerGymDataCollator:
    return_tensors: str = "pt"
    max_len: int = 20 #subsets of the episode we use for training
    state_dim: int = 17  # size of state space
    act_dim: int = 6  # size of action space
    max_ep_len: int = 1000 # max episode length in the dataset
    scale: float = 1000.0  # normalization of rewards/returns
    state_mean: np.array = None  # to store state means
    state_std: np.array = None  # to store state stds
    p_sample: np.array = None  # a distribution to take account trajectory lengths
    n_traj: int = 0 # to store the number of trajectories in the dataset

    def __init__(self, dataset) -> None:
        self.act_dim = len(dataset[0]["actions"][0])
        self.state_dim = len(dataset[0]["observations"][0])
        self.dataset = dataset
        # calculate dataset stats for normalization of states
        states = []
        traj_lens = []
        for obs in dataset["observations"]:
            states.extend(obs)
            traj_lens.append(len(obs))
        self.n_traj = len(traj_lens)
        states = np.vstack(states)
        self.state_mean, self.state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6

        traj_lens = np.array(traj_lens)
        self.p_sample = traj_lens / sum(traj_lens)

    def _discount_cumsum(self, x, gamma):
        discount_cumsum = np.zeros_like(x)
        discount_cumsum[-1] = x[-1]
        for t in reversed(range(x.shape[0] - 1)):
            discount_cumsum[t] = x[t] + gamma * discount_cumsum[t + 1]
        return discount_cumsum

    def __call__(self, features):
        batch_size = len(features)
        # this is a bit of a hack to be able to sample of a non-uniform distribution
        batch_inds = np.random.choice(
            np.arange(self.n_traj),
            size=batch_size,
            replace=True,
            p=self.p_sample,  # reweights so we sample according to timesteps
        )
        # a batch of dataset features
        s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []

        for ind in batch_inds:
            # for feature in features:
            feature = self.dataset[int(ind)]
            si = random.randint(0, len(feature["rewards"]) - 1)

            # get sequences from dataset
            s.append(np.array(feature["observations"][si : si + self.max_len]).reshape(1, -1, self.state_dim))
            a.append(np.array(feature["actions"][si : si + self.max_len]).reshape(1, -1, self.act_dim))
            r.append(np.array(feature["rewards"][si : si + self.max_len]).reshape(1, -1, 1))

            d.append(np.array(feature["dones"][si : si + self.max_len]).reshape(1, -1))
            timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
            timesteps[-1][timesteps[-1] >= self.max_ep_len] = self.max_ep_len - 1  # padding cutoff
            rtg.append(
                self._discount_cumsum(np.array(feature["rewards"][si:]), gamma=1.0)[
                    : s[-1].shape[1]
                ].reshape(1, -1, 1)
            )
            if rtg[-1].shape[1] < s[-1].shape[1]:
                print("if true")
                rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

            # padding and state + reward normalization
            tlen = s[-1].shape[1]
            s[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, self.state_dim)), s[-1]], axis=1)
            s[-1] = (s[-1] - self.state_mean) / self.state_std
            a[-1] = np.concatenate(
                [np.ones((1, self.max_len - tlen, self.act_dim)) * -10.0, a[-1]],
                axis=1,
            )
            r[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), r[-1]], axis=1)
            d[-1] = np.concatenate([np.ones((1, self.max_len - tlen)) * 2, d[-1]], axis=1)
            rtg[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), rtg[-1]], axis=1) / self.scale
            timesteps[-1] = np.concatenate([np.zeros((1, self.max_len - tlen)), timesteps[-1]], axis=1)
            mask.append(np.concatenate([np.zeros((1, self.max_len - tlen)), np.ones((1, tlen))], axis=1))

        s = torch.from_numpy(np.concatenate(s, axis=0)).float()
        a = torch.from_numpy(np.concatenate(a, axis=0)).float()
        r = torch.from_numpy(np.concatenate(r, axis=0)).float()
        d = torch.from_numpy(np.concatenate(d, axis=0))
        rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).float()
        timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).long()
        mask = torch.from_numpy(np.concatenate(mask, axis=0)).float()

        return {
            "states": s,
            "actions": a,
            "rewards": r,
            "returns_to_go": rtg,
            "timesteps": timesteps,
            "attention_mask": mask,
        }

### Extending the Decision Transformer Model to include a loss function

In order to train the model with the HuggingFace trainer class, we first need to ensure the dictionary it returns contains a loss, in this case L-2 norm of the models action predictions and the targets.
The L-2 norm loss is given by:

$$\text{Loss} = \| \hat{a} - a \|_2 = \sqrt{\sum_{i=1}^{n} (\hat{a}_i - a_i)^2}$$

In [ ]:
class TrainableDT(DecisionTransformerModel):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, **kwargs):
        output = super().forward(**kwargs)
        # add the DT loss
        action_preds = output[1]
        action_targets = kwargs["actions"]
        attention_mask = kwargs["attention_mask"]
        act_dim = action_preds.shape[2]
        action_preds = action_preds.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
        action_targets = action_targets.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]

        loss = torch.mean((action_preds - action_targets) ** 2)

        return {"loss": loss}

    def original_forward(self, **kwargs):
        return super().forward(**kwargs)

In [ ]:
collator = DecisionTransformerGymDataCollator(dataset["train"])

config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim)
model = TrainableDT(config)

In [ ]:
print(model)

**Model Architecture**

1. Encoder (DecisionTransformerGPT2Model):
This is the core of the model, based on the GPT-2 architecture but adapted for decision-making tasks. It includes:

 * Token and position embeddings (wte and wpe)
 * A dropout layer for regularization
 * A series of transformer blocks (3 in this case)
 * A final layer normalization (ln_f)

  Each transformer block contains:

 * Two layer normalization layers (ln_1 and ln_2)
 * A self-attention mechanism (DecisionTransformerGPT2Attention)
 * A feedforward neural network (DecisionTransformerGPT2MLP)


2. Embedding layers:

 * embed_timestep: Embeds the timestep information
 * embed_return: Transforms the return (reward) into an embedding
 * embed_state: Transforms the state into an embedding
 * embed_action: Transforms the action into an embedding
 * embed_ln: A layer normalization applied to the combined embeddings


3. Prediction layers:

 * predict_state: Predicts the next state
 * predict_action: Predicts the next action (with a tanh activation)
 * predict_return: Predicts the expected return



The model works by first embedding the input sequences (state, action, return, and timestep) into a common representation space. These embeddings are then processed through the transformer encoder, which allows the model to attend to different parts of the input sequence. Finally, the output of the encoder is used to make predictions about the next state, action, and expected return.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nTotal trainable parameters: {count_parameters(model)}")

# You can also print the config
print("\nModel Configuration:")
print(config)

Here we have a view of the model parameters.

### Defining the training hyperparameters and training the model
Here, we define the training hyperparameters and our Trainer class that we'll use to train our Decision Transformer model.

This step takes about an hour, so you may leave it running. It must be noted that authors have trained for at least 3 hours, so the results presented here are not as performant as the models hosted on the HuggingFace hub, but performance still quite good.

In [ ]:
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=120,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=collator,
)

trainer.train()

### Visualize the performance of the agent

In this last part we will see how agent performs and interact with environment.

In [ ]:
import mujoco_py
import gym

from colabgymrender.recorder import Recorder

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model from checkpoint
checkpoint_path = "/content/output/checkpoint-1500"
config = DecisionTransformerConfig.from_pretrained(checkpoint_path)
model = DecisionTransformerModel.from_pretrained(checkpoint_path, config=config)
model.to(device)

In [ ]:
def get_action(model, states, actions, rewards, returns_to_go, timesteps):
    # Get the device of the model
    device = next(model.parameters()).device

    # Move all inputs to the same device as the model
    states = states.to(device)
    actions = actions.to(device)
    returns_to_go = returns_to_go.to(device)
    timesteps = timesteps.to(device)

    states = states.reshape(1, -1, model.config.state_dim)
    actions = actions.reshape(1, -1, model.config.act_dim)
    returns_to_go = returns_to_go.reshape(1, -1, 1)
    timesteps = timesteps.reshape(1, -1)

    states = states[:, -model.config.max_length :]
    actions = actions[:, -model.config.max_length :]
    returns_to_go = returns_to_go[:, -model.config.max_length :]
    timesteps = timesteps[:, -model.config.max_length :]
    padding = model.config.max_length - states.shape[1]

    # pad all tokens to sequence length
    attention_mask = torch.cat([torch.zeros(padding, device=device), torch.ones(states.shape[1], device=device)])
    attention_mask = attention_mask.to(dtype=torch.long, device=device).reshape(1, -1)
    states = torch.cat([torch.zeros((1, padding, model.config.state_dim), device=device), states], dim=1).float()
    actions = torch.cat([torch.zeros((1, padding, model.config.act_dim), device=device), actions], dim=1).float()
    returns_to_go = torch.cat([torch.zeros((1, padding, 1), device=device), returns_to_go], dim=1).float()
    timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long, device=device), timesteps], dim=1)

    output = model(
        states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=returns_to_go,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=True,
    )

    return output.action_preds[0, -1]

In [ ]:
# build the environment
directory = './video'
model = model.to(device)
env = gym.make("HalfCheetah-v3")
env = Recorder(env, directory, fps=30)
max_ep_len = 1000
scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 12000 / scale  # evaluation is conditioned on a return of 12000, scaled accordingly

state_mean = collator.state_mean.astype(np.float32)
state_std = collator.state_std.astype(np.float32)
print(state_mean)

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

# Create the decision transformer model
state_mean = torch.from_numpy(state_mean).to(device=device)
state_std = torch.from_numpy(state_std).to(device=device)


In [ ]:
import numpy as np
import torch

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Number of episodes to run
num_episodes = 10

# List to store episode returns
episode_returns = []

for episode in range(num_episodes):
    episode_return, episode_length = 0, 0
    state = env.reset()
    target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
    states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
    actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
    rewards = torch.zeros(0, device=device, dtype=torch.float32)

    timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
    for t in range(max_ep_len):
        actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
        rewards = torch.cat([rewards, torch.zeros(1, device=device)])

        action = get_action(
            model,
            (states - state_mean) / state_std,
            actions,
            rewards,
            target_return,
            timesteps,
        )
        actions[-1] = action
        action = action.detach().cpu().numpy()

        state, reward, done, _ = env.step(action)

        cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
        states = torch.cat([states, cur_state], dim=0)
        rewards[-1] = reward

        pred_return = target_return[0, -1] - (reward / scale)
        target_return = torch.cat([target_return, pred_return.reshape(1, 1)], dim=1)
        timesteps = torch.cat([timesteps, torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

        episode_return += reward.item()  # Use .item() to get the scalar value
        episode_length += 1

        if done:
            break

    # Append the episode return to our list
    episode_returns.append(episode_return)

# Calculate mean and standard deviation of returns
mean_reward = np.mean(episode_returns)
std_reward = np.std(episode_returns)

# Print the performance
print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Play the video
env.play()

In [ ]:
from google.colab import files
import shutil

# Zip the folder
shutil.make_archive('/content/checkpoint_folder', 'zip', '/content/output')

# Download the zip file
files.download('/content/checkpoint_folder.zip')

# Zip the folder
shutil.make_archive('/content/DT_video', 'zip', '/content/video')
files.download('/content/DT_video.zip')
files.download('/content/__temp__.mp4')


In [ ]:
!pip freeze > requirements_DT.txt